In [26]:
# import dependencies
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine
from config import dbConnectionString
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
engine = create_engine(dbConnectionString)

#import dataframe from SQL database
df = pd.read_sql_query('select * from "cardioData"',con=engine)
df = df.drop(columns = ['index', 'id'])
df

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [3]:
le = LabelEncoder()
df2 = df.copy()
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [4]:
df2['BMI'] = df2['BMI'].astype(float)

df2['BMI_class'] = ''

for i in range(len(df2['age'])):
    if df2['BMI'][i] < 18.5:
        df2['BMI_class'][i] = 'underweight'
    elif df2['BMI'][i] < 24.9 and df2['BMI'][i] > 18.5:
        df2['BMI_class'][i] = 'normal'
    elif df2['BMI'][i] < 29.9 and df2['BMI'][i] > 24.9:
        df2['BMI_class'][i] = 'overweight'
    elif df2['BMI'][i] < 34.9 and df2['BMI'][i] > 29.9:
        df2['BMI_class'][i] = 'obese'
    elif df2['BMI'][i] < 39.9 and df2['BMI'][i] > 34.9:
        df2['BMI_class'][i] = 'obese2'
    else:
        df2['BMI_class'][i] = 'obese3'
        
df2['ap_hi_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_hi'][i] < 120:
        df2['ap_hi_class'][i] = 'normal'
    elif df2['ap_hi'][i] < 129 and df2['ap_hi'][i] >= 120:
        df2['ap_hi_class'][i] = 'elevated'
    elif df2['ap_hi'][i] < 139 and df2['ap_hi'][i] >= 130:
        df2['ap_hi_class'][i] = 'highbloodpressure'
    elif df2['ap_hi'][i] < 180 and df2['ap_hi'][i] >= 140:
        df2['ap_hi_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_hi_class'][i] = 'hypertensive'
        
df2['ap_lo_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_lo'][i] < 80:
        df2['ap_lo_class'][i] = 'elevated'
    elif df2['ap_lo'][i] < 80 and df2['ap_lo'][i] >= 89:
        df2['ap_lo_class'][i] = 'highbloodpressure'
    elif df2['ap_lo'][i] < 120 and df2['ap_lo'][i] >= 90:
        df2['ap_lo_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_lo_class'][i] = 'hypertensive'
        
df2

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShell

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,Female,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,Male,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,Male,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,Male,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,Male,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,Male,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,Female,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,Male,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [5]:
#female = 0, male = 1
df2['gender'] = le.fit_transform(df2['gender'])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,1,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,1,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,1,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,1,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,1,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,0,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,1,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [6]:
normal_dic = {
    "normal" : 0,
    "above normal" : 1,
    "well above normal" : 2,
}

true_false_dic = {
    True : 1,
    False : 0,
}

BMI_dic = {
    "underweight" : 0,
    "normal" : 1,
    "overweight" : 2, 
    "obese": 3, 
    "obese2" : 4, 
    "obese3" : 5
}

blood_pressure_dic = {
    "normal" : 0,
    "elevated" : 1, 
    "highbloodpressure": 2, 
    "highbloodpressure2": 3,
    "hypertensive" : 4
}

df2['cardiovascular'] = df2['cardiovascular'].apply(lambda x : true_false_dic[x])
df2['smoke'] = df2['smoke'].apply(lambda x : true_false_dic[x])
df2['alchohol'] = df2['alchohol'].apply(lambda x : true_false_dic[x])
df2['active'] = df2['active'].apply(lambda x : true_false_dic[x])

df2['cholestrol'] = df2['cholestrol'].apply(lambda x : normal_dic[x])
df2['gloucose'] = df2['gloucose'].apply(lambda x : normal_dic[x])
df2['BMI_class'] = df2['BMI_class'].apply(lambda x : BMI_dic[x])
df2['ap_hi_class'] = df2['ap_hi_class'].apply(lambda x : blood_pressure_dic[x])
df2['ap_lo_class'] = df2['ap_lo_class'].apply(lambda x : blood_pressure_dic[x])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,0,23.0,110,80,0,0,0,0,1,1,0,4
1,55.4,1,156.0,85.0,1,46.0,140,90,2,0,0,0,1,5,3,3
2,51.7,1,165.0,64.0,1,25.0,130,70,2,0,0,0,0,2,2,1
3,47.9,1,156.0,56.0,0,20.0,100,60,0,0,0,0,0,1,0,1
4,60.0,1,151.0,67.0,0,30.0,120,80,1,1,0,0,0,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,0,34.0,120,80,0,0,1,0,1,3,1,4
63912,61.9,1,158.0,126.0,1,100.0,140,90,1,1,0,0,1,5,3,3
63913,52.2,0,183.0,105.0,1,60.0,180,90,2,0,0,1,0,5,4,3
63914,61.5,1,163.0,72.0,1,32.0,135,80,0,2,0,0,0,3,2,4


In [16]:
#neural network1

X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular'].values

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3

nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 13)                156       
_________________________________________________________________
dense_49 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_50 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_51 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_52 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 4         
Total params: 443
Trainable params: 443
Non-trainable params: 0
_______________________________________________________

In [17]:
#using BMI

X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3

nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 13)                143       
_________________________________________________________________
dense_55 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_57 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_58 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 4         
Total params: 430
Trainable params: 430
Non-trainable params: 0
_______________________________________________________

In [18]:
#bucketing BMI

X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3

nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 13)                143       
_________________________________________________________________
dense_61 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_62 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_63 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_64 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 4         
Total params: 430
Trainable params: 430
Non-trainable params: 0
_______________________________________________________

Loss: 0.5463755130767822, Accuracy: 0.7295200228691101


In [19]:
#bucketing blood pressure

X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3

nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 13)                156       
_________________________________________________________________
dense_67 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_68 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_69 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_70 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 4         
Total params: 443
Trainable params: 443
Non-trainable params: 0
_______________________________________________________

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/67
1499/1499 [==============================] - 1s 560us/step - loss: 0.5840 - accuracy: 0.6961
Epoch 2/67
1499/1499 [==============================] - 1s 536us/step - loss: 0.5579 - accuracy: 0.7209
Epoch 3/67
1499/1499 [==============================] - 1s 533us/step - loss: 0.5557 - accuracy: 0.7223
Epoch 4/67
1499/1499 [==============================] - 1s 530us/step - loss: 0.5545 - accuracy: 0.7233
Epoch 5/67
1499/1499 [==============================] - 1s 532us/step - loss: 0.5539 - accuracy: 0.7222
Epoch 6/67
1499/1499 [==============================] - 1s 535us/step - loss: 0.5531 - accuracy: 0.7233
Epoch 7/67
1499/1499 [==============================] - 1s 539us/step - loss: 0.5528 - accuracy: 0.7238
Epoch 8/67
1499/1499 [==============================] - 1s 537us/step - loss: 0.5525 - accuracy: 0.7237
Epoch 9/67
1499/1499 [==============================] - 1s 533us/step - loss: 0.5521 - accuracy: 0.7240
Epoch 10/67
1499/1499 [==============================] - 1s 534u

In [20]:
#bucketing both

X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3

nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 13)                143       
_________________________________________________________________
dense_73 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_74 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_75 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_76 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_77 (Dense)             (None, 1)                 4         
Total params: 430
Trainable params: 430
Non-trainable params: 0
_______________________________________________________

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


1499/1499 [==============================] - 1s 565us/step - loss: 0.5794 - accuracy: 0.7037
Epoch 2/67
1499/1499 [==============================] - 1s 535us/step - loss: 0.5569 - accuracy: 0.7215
Epoch 3/67
1499/1499 [==============================] - 1s 531us/step - loss: 0.5548 - accuracy: 0.7236
Epoch 4/67
1499/1499 [==============================] - 1s 529us/step - loss: 0.5542 - accuracy: 0.7240
Epoch 5/67
1499/1499 [==============================] - 1s 547us/step - loss: 0.5539 - accuracy: 0.7243
Epoch 6/67
1499/1499 [==============================] - 1s 531us/step - loss: 0.5534 - accuracy: 0.7235
Epoch 7/67
1499/1499 [==============================] - 1s 532us/step - loss: 0.5527 - accuracy: 0.7247
Epoch 8/67
1499/1499 [==============================] - 1s 528us/step - loss: 0.5524 - accuracy: 0.7244
Epoch 9/67
1499/1499 [==============================] - 1s 535us/step - loss: 0.5521 - accuracy: 0.7247
Epoch 10/67
1499/1499 [==============================] - 1s 531us/step - lo

## Using BMI and no Bucketing provides the best result

In [41]:
#using BMI

X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3



nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="sigmoid"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="sigmoid"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath= 'checkpoints/weights.{epoch:02d}.hdf5',
    verbose=1,
    save_weights_only=True,
    save_freq=4020)
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=67, callbacks = [cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn_model.save('cardio_model.h5')
nn_model.save_weights('nn_model/nn_model.h5')
model_json = nn_model.to_json()
with open("nn_model/nn_model.json", "w") as json_file:
    json_file.write(model_json)
    
# Save scalar info
means = scaler.mean_
var = scaler.var_
np.save('nn_model/scaler_means', means)
np.save('nn_model/scalar_var', var)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_200 (Dense)            (None, 13)                143       
_________________________________________________________________
dense_201 (Dense)            (None, 8)                 112       
_________________________________________________________________
dense_202 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_203 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_204 (Dense)            (None, 3)                 27        
_________________________________________________________________
dense_205 (Dense)            (None, 1)                 4         
Total params: 430
Trainable params: 430
Non-trainable params: 0
_______________________________________________________

In [39]:
#using BMI and dropping low features

X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class', 'smoke', 'active', 'alchohol'], 1).values
y = df2['cardiovascular']

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3



nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="sigmoid"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="sigmoid"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath= 'checkpoints/weights.{epoch:02d}.hdf5',
    verbose=1,
    save_weights_only=True,
    save_freq=4020)
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=200, callbacks = [cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn_model.save('cardio_model.h5')
nn_model.save_weights('nn_model/nn_model.h5')
model_json = nn_model.to_json()
with open("nn_model/nn_model.json", "w") as json_file:
    json_file.write(model_json)
    
# Save scalar info
means = scaler.mean_
var = scaler.var_
np.save('nn_model/scaler_means', means)
np.save('nn_model/scalar_var', var)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_188 (Dense)            (None, 13)                104       
_________________________________________________________________
dense_189 (Dense)            (None, 8)                 112       
_________________________________________________________________
dense_190 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_191 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_192 (Dense)            (None, 3)                 27        
_________________________________________________________________
dense_193 (Dense)            (None, 1)                 4         
Total params: 391
Trainable params: 391
Non-trainable params: 0
_______________________________________________________

1499/1499 [==============================] - 1s 664us/step - loss: 0.5521 - accuracy: 0.7245
Epoch 97/200
 807/1499 [===============>..............] - ETA: 0s - loss: 0.5554 - accuracy: 0.7213
Epoch 00097: saving model to checkpoints\weights.97.hdf5
1499/1499 [==============================] - 1s 822us/step - loss: 0.5522 - accuracy: 0.7244
Epoch 98/200
1499/1499 [==============================] - 2s 1ms/step - loss: 0.5521 - accuracy: 0.7241
Epoch 99/200
1499/1499 [==============================] - 1s 876us/step - loss: 0.5520 - accuracy: 0.7247
Epoch 100/200
 332/1499 [=====>........................] - ETA: 1s - loss: 0.5524 - accuracy: 0.7241
Epoch 00100: saving model to checkpoints\weights.100.hdf5
1499/1499 [==============================] - 1s 762us/step - loss: 0.5521 - accuracy: 0.7247
Epoch 101/200
1499/1499 [==============================] - 1s 990us/step - loss: 0.5520 - accuracy: 0.7236
Epoch 102/200
1315/1499 [=========================>....] - ETA: 0s - loss: 0.5504 - accu

1499/1499 [==============================] - 1s 571us/step - loss: 0.5514 - accuracy: 0.7239
Epoch 148/200
 733/1499 [=============>................] - ETA: 0s - loss: 0.5555 - accuracy: 0.7200
Epoch 00148: saving model to checkpoints\weights.148.hdf5
1499/1499 [==============================] - 1s 572us/step - loss: 0.5512 - accuracy: 0.7251
Epoch 149/200
1499/1499 [==============================] - 1s 613us/step - loss: 0.5514 - accuracy: 0.7240
Epoch 150/200
1499/1499 [==============================] - 1s 551us/step - loss: 0.5514 - accuracy: 0.7243
Epoch 151/200
 187/1499 [==>...........................] - ETA: 0s - loss: 0.5504 - accuracy: 0.7268
Epoch 00151: saving model to checkpoints\weights.151.hdf5
1499/1499 [==============================] - 1s 553us/step - loss: 0.5514 - accuracy: 0.7246
Epoch 152/200
1499/1499 [==============================] - 1s 550us/step - loss: 0.5513 - accuracy: 0.7234
Epoch 153/200
1262/1499 [========================>.....] - ETA: 0s - loss: 0.5518 